# Eliminación de anomalías de una imagen

## Introducción

## Librerías a utilizar

In [ ]:
import cv2 as cv
import numpy as np

## Carga de imágenes

In [ ]:
image_unir = cv.imread('im/unir-1.jpg')
image_rgb = cv.imread('im/rgb.png')